In [1]:
DATA_NAME = 'segmentAnything-full' 
TRANSFORM = 'learned'
CHANNEL = ''

In [2]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_learned_full_segmentAnything.ipynb


In [3]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [4]:
from scipy import spatial
from sklearn.decomposition import PCA

In [5]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))

In [6]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)
elif 'learned' in TRANSFORM:
    nonskewed_df = pd.read_csv(os.path.join(ROOT_DIR, 'learned-filters', 'nonskewed_filter_idxs_df.csv')).set_index(['dataset', 'num_images', 'num_bootstrap'])
    nonskewed_filter_idxs = eval(nonskewed_df.loc[DATA_NAME].sort_values('num_images', ascending=False)['nonskewed_filter_idxs'].iloc[0]) 
    GROUPS = nonskewed_filter_idxs # can set to filter_group_map.keys() to include all prepared filters

In [7]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [8]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
    cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [9]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

,0,1,2,3,4,5,6,7,8,9,...,54,55,56,57,58,59,60,61,62,63
0,6.04,0.00,-0.00,-0.00,0.00,0.0,0.0,0.0,0.0,-0.0,...,-0.0,0.0,0.0,-0.0,0.0,0.00,-0.00,-0.00,0.00,0.00
1,0.00,6.45,-0.00,-0.00,-0.00,0.0,0.0,0.0,-0.0,0.0,...,-0.0,-0.0,-0.0,-0.0,-0.0,0.00,-0.00,0.00,0.00,-0.00
2,-0.00,-0.00,7.66,0.00,0.00,-0.0,0.0,-0.0,-0.0,-0.0,...,-0.0,-0.0,-0.0,0.0,0.0,-0.00,0.00,-0.00,0.00,0.00
3,-0.00,-0.00,0.00,5.17,-0.00,-0.0,0.0,0.0,0.0,-0.0,...,-0.0,0.0,0.0,0.0,-0.0,0.00,0.00,-0.00,-0.00,-0.00
4,0.00,-0.00,0.00,-0.00,9.39,-0.0,-0.0,0.0,0.0,0.0,...,-0.0,0.0,0.0,0.0,0.0,-0.00,-0.00,0.00,0.00,-0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,0.00,0.00,-0.00,0.00,-0.00,0.0,0.0,-0.0,0.0,0.0,...,0.0,-0.0,0.0,0.0,-0.0,7.43,0.00,-0.00,-0.00,-0.00
60,-0.00,-0.00,0.00,0.00,-0.00,-0.0,-0.0,-0.0,0.0,0.0,...,-0.0,-0.0,0.0,0.0,-0.0,0.00,0.75,-0.00,0.00,0.00
61,-0.00,0.00,-0.00,-0.00,0.00,-0.0,-0.0,-0.0,-0.0,-0.0,...,-0.0,-0.0,-0.0,-0.0,0.0,-0.00,-0.00,3.19,0.00,0.00
62,0.00,0.00,0.00,-0.00,0.00,-0.0,-0.0,0.0,0.0,-0.0,...,-0.0,-0.0,-0.0,0.0,-0.0,-0.00,0.00,0.00,5.73,-0.00


In [10]:
corr_matrix = cov_matrix / np.sqrt(np.outer(np.diag(cov_matrix), np.diag(cov_matrix)))
corr_df = pd.DataFrame(corr_matrix, index=GROUPS, columns=GROUPS)
corr_df.round(5)

,0,1,2,3,4,5,6,7,8,9,...,54,55,56,57,58,59,60,61,62,63
0,1.00000,0.00001,-0.00002,-0.00002,0.00003,0.00001,0.00001,0.00002,0.00001,-0.00003,...,-0.00001,0.00002,0.00000,-0.00005,0.00002,0.00000,-0.00002,-0.00001,0.00002,0.00000
1,0.00001,1.00000,-0.00002,-0.00002,-0.00000,0.00000,0.00004,0.00005,-0.00000,0.00002,...,-0.00002,-0.00004,-0.00001,-0.00006,-0.00002,0.00005,-0.00002,0.00003,0.00004,-0.00003
2,-0.00002,-0.00002,1.00000,0.00004,0.00004,-0.00003,0.00005,-0.00004,-0.00002,-0.00002,...,-0.00001,-0.00000,-0.00002,0.00003,0.00001,-0.00003,0.00001,-0.00003,0.00005,0.00002
3,-0.00002,-0.00002,0.00004,1.00000,-0.00003,-0.00006,0.00001,0.00002,0.00003,-0.00007,...,-0.00002,0.00002,0.00004,0.00002,-0.00002,0.00002,0.00001,-0.00005,-0.00000,-0.00003
4,0.00003,-0.00000,0.00004,-0.00003,1.00000,-0.00004,-0.00000,0.00001,0.00002,0.00001,...,-0.00001,0.00001,0.00001,0.00001,0.00002,-0.00002,-0.00002,0.00002,0.00005,-0.00001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,0.00000,0.00005,-0.00003,0.00002,-0.00002,0.00000,0.00001,-0.00001,0.00001,0.00002,...,0.00004,-0.00003,0.00001,0.00000,-0.00007,1.00000,0.00001,-0.00000,-0.00002,-0.00001
60,-0.00002,-0.00002,0.00001,0.00001,-0.00002,-0.00003,-0.00001,-0.00003,0.00002,0.00001,...,-0.00005,-0.00000,0.00001,0.00004,-0.00004,0.00001,1.00000,-0.00001,0.00004,0.00002
61,-0.00001,0.00003,-0.00003,-0.00005,0.00002,-0.00002,-0.00001,-0.00002,-0.00002,-0.00004,...,-0.00001,-0.00004,-0.00001,-0.00001,0.00000,-0.00000,-0.00001,1.00000,0.00002,0.00002
62,0.00002,0.00004,0.00005,-0.00000,0.00005,-0.00007,-0.00005,0.00005,0.00005,-0.00000,...,-0.00003,-0.00004,-0.00001,0.00001,-0.00000,-0.00002,0.00004,0.00002,1.00000,-0.00001


In [11]:
np.linalg.norm(cov_matrix - np.diag(np.diag(cov_matrix)))  # Remove diagonal elements for visualization

0.010362793365876305

In [12]:
pca = PCA()
pca.fit(cov_matrix)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[6.08851466e+00 4.81084861e+00 4.05857473e+00 1.59632772e+00
 1.38351124e+00 9.84761732e-01 9.30153627e-01 9.13231621e-01
 8.64672798e-01 7.49443618e-01 7.08461351e-01 6.79364563e-01
 6.66853653e-01 6.58113428e-01 6.40036628e-01 6.05927665e-01
 5.75703257e-01 5.61593179e-01 5.42458773e-01 5.31532537e-01
 5.10577171e-01 4.82962210e-01 4.66596973e-01 4.41160399e-01
 4.17158419e-01 3.95202309e-01 3.64286715e-01 3.49831543e-01
 3.42210674e-01 3.34682209e-01 3.17197500e-01 2.84143613e-01
 2.76846709e-01 2.63867708e-01 2.49528691e-01 2.38779452e-01
 2.36519178e-01 2.28313955e-01 2.16335392e-01 2.14024662e-01
 2.07241436e-01 1.98147666e-01 1.88611498e-01 1.79888638e-01
 1.71230152e-01 1.68071956e-01 1.62991484e-01 1.61640145e-01
 1.61344418e-01 1.57343977e-01 1.51049103e-01 1.46105926e-01
 1.41804804e-01 1.31074512e-01 1.24136202e-01 1.14051800e-01
 1.09158307e-01 1.05552313e-01 9.05151058e-02 8.65597036e-02
 4.09466129e-02 1.36136979e-02 4.10934073e-03 3

,0,1,2,3,4,5,6,7,8,9,...,54,55,56,57,58,59,60,61,62,63
0,-0.004582,-0.005029,-0.006185,-0.003837,-0.008379,-0.004435,-0.002312,-0.002942,-0.000247,-0.001957,...,-0.003062,-0.002428,-0.004908,-0.002624,-0.002249,-0.005991,-0.000525,-0.002238,-0.004342,-0.001621
1,-0.005542,-0.006066,-0.007711,-0.004567,-0.010705,-0.005384,-0.002735,-0.003513,-0.000286,-0.002290,...,-0.003663,-0.002828,-0.006012,-0.003121,-0.002676,-0.007318,-0.000608,-0.002669,-0.005186,-0.001932
2,-0.007083,-0.007735,-0.009995,-0.005799,-0.014439,-0.006801,-0.003451,-0.004460,-0.000361,-0.002874,...,-0.004609,-0.003571,-0.007703,-0.003937,-0.003344,-0.009541,-0.000754,-0.003356,-0.006614,-0.002382
3,-0.010812,-0.012746,-0.020901,-0.008012,-0.087627,-0.010081,-0.004198,-0.005673,-0.000407,-0.003447,...,-0.005917,-0.004338,-0.012377,-0.004894,-0.004039,-0.018994,-0.000851,-0.004033,-0.009873,-0.002805
4,-0.013846,-0.016551,-0.031175,-0.010018,0.992175,-0.012950,-0.005009,-0.006817,-0.000475,-0.004069,...,-0.007167,-0.005224,-0.016290,-0.005862,-0.004803,-0.027244,-0.001019,-0.004812,-0.012220,-0.003332
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,-0.000251,-0.000224,-0.000148,-0.000239,-0.000160,-0.000214,-0.000891,-0.000495,0.000090,-0.001394,...,-0.000430,-0.000742,-0.000228,-0.000577,-0.000807,-0.000153,0.000231,-0.000878,-0.000292,0.716713
60,-0.025322,-0.023471,-0.019374,-0.030426,-0.015579,-0.026279,-0.057034,-0.040765,0.020684,-0.077067,...,-0.039108,-0.053856,-0.023578,-0.047308,-0.059735,-0.020072,0.053006,-0.059698,-0.026891,-0.115410
61,-0.047948,-0.044795,-0.037464,-0.056598,-0.030439,-0.049661,-0.093786,-0.072497,0.138503,-0.114754,...,-0.070006,-0.090028,-0.044983,-0.081625,-0.096963,-0.038685,0.731146,-0.096868,-0.050709,-0.143933
62,-0.039367,-0.036851,-0.030944,-0.046164,-0.025239,-0.040734,-0.073822,-0.058348,0.642875,-0.088072,...,-0.056498,-0.071156,-0.036986,-0.065062,-0.076064,-0.031934,-0.578275,-0.075973,-0.041530,-0.106248


In [13]:
cos_dist = spatial.distance.cdist(eigenvectors, np.eye(len(GROUPS)), metric='cosine')
cos_dist = [np.min(row) for row in cos_dist]
cos_dist


[0.0028913743261660407,
 0.006436504448510627,
 0.00568607876452043,
 0.006350121909672657,
 0.007824850459269617,
 0.02202873465583388,
 0.23943275877413395,
 0.2821994194845022,
 0.056099161292478494,
 0.027933602764735666,
 0.044534104892096305,
 0.0863351632800956,
 0.1613880739360235,
 0.294450704018552,
 0.36393337261368797,
 0.09657205609585251,
 0.12716011406477223,
 0.17517249211870356,
 0.2884859361225134,
 0.4078631893118786,
 0.2544521690349999,
 0.19179605289552049,
 0.2640857493515606,
 0.1575851571631608,
 0.14683342988259174,
 0.13364794945738778,
 0.08708641506448678,
 0.1455883080924264,
 0.25517194838418766,
 0.2838126564022698,
 0.3813342655867582,
 0.17877024916455275,
 0.26278981031248105,
 0.1539529947923498,
 0.11231085105406102,
 0.16629098653645313,
 0.26950954922907433,
 0.47421889177337206,
 0.193805269356905,
 0.25488223672660115,
 0.32498109967205724,
 0.35428703626845315,
 0.25157376511857543,
 0.20070189581310416,
 0.22341063647861903,
 0.337533207942735